In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')
df_new = pd.DataFrame()

data_time_format = "%d-%m月 -%y %H.%M.%S.%f"

# 拆分出时间和上下午
df_new['日期时间'] = pd.to_datetime(df['操作时间'].str.rsplit(' ', 1, expand=True)[0], format=data_time_format)

# 添加12小时给下午的日期时间
pm_rows = df['操作时间'].str.endswith("下午")
df_new.loc[pm_rows, '日期时间'] += pd.to_timedelta(12, unit='h')

df_new['账号'] = df['从账号'].copy()
df_new['机构'] = df['机构'].copy()

df_new['操作内容'] = df['操作内容'].str.slice(36, None)



df_new['源IP'] = df['源IP'].copy()

# del df

df_new


/var/folders/4q/6dp0mwtx3ns522q3t7kkc6yc0000gn/T/ipykernel_65202/883271930.py:7: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df_new['日期时间'] = pd.to_datetime(df['操作时间'].str.rsplit(' ', 1, expand=True)[0], format=data_time_format)


,日期时间,账号,机构,操作内容,源IP
0,2023-05-31 23:01:44,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问智能家居日报表模块,10.136.14.86
1,2023-05-31 23:01:00,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问智能家居质量日报模块,10.136.14.86
2,2023-05-31 23:00:52,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问首页模块,10.136.14.86
3,2023-05-31 23:00:51,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,用户登陆,10.136.14.86
4,2023-05-31 23:00:10,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,用户登陆,10.136.14.86
...,...,...,...,...,...
109864,2023-05-01 08:22:54,1c7cc983-5870-4892-af7b-c06a86ce59d7,延安全市,用户登陆,10.136.50.122
109865,2023-05-01 08:22:55,1c7cc983-5870-4892-af7b-c06a86ce59d7,延安全市,访问首页模块,10.136.50.122
109866,2023-05-01 08:21:08,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,访问5GSA驻留比模块,10.136.71.74
109867,2023-05-01 08:21:02,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,访问首页模块,10.136.71.74


数据预处理：
1. 上午时间不变，下午的时间+12
2. 时间里的毫秒去掉
3. 时间读取到pandas内 dates = pd.to_datetime(date_strings, format="%d-%m月 -%y %H.%M.%S.%f")
4. 因为数据只给了同一个月份的，暂且先不考虑跨月和跨年，先考虑跨日
5. 操作内容前缀去掉

In [3]:
class DataProcess:
    def __init__(self, data: pd.DataFrame) -> None:
        self.data = data
    
    def process(self):
        self.data = self.data.sort_values('日期时间')
        return self.data


分别定义五个类：

业务高频访问：为每一个业务维护一个字典，{账户:访问次数, 账户:访问次数……}，存储不同账号访问的次数，每隔1分钟清零，字典值超过5打上业务高频访问标签。

账号复用：为每一个账号维护一个集合，保存此账号每次登录时的ip和时间，两分钟内如果集合len>=3，即账号复用，每隔2分钟清零。

非常用IP访问：TODO:这个是否要一次读完全部日志？一条一条读无法判断之后还会不会来。

非工作时间访问：不在8:00~19:00之间的日志都是。

登录异常： 为每个账号维护一个字典，记录访问次数，字典值>=8打上登录异常，一分钟清零

         为每个账号维护一个集合，保存此账号登录的ip，一分钟内若集合len>=2视为登录异常

---


清零操作：每次读到一行数据，如果是一分钟清零，如果新数据的分钟数 = 原来数据分钟数+1，将原数据结构清零。

         两分钟清零就是如果读到了新数据分钟数 = min(原始数据分钟数)+2



优化空间：
1. 五个操作共同维护数据结构，{账号1:{访问次数:{}}, 账号2：{}} 
2. 登录异常1分钟2个ip和2分钟三个ip的逻辑能不能合在一起，只用一个字典来处理。
3. 

为了减少损耗，一次读取的时候同时进行五个操作，因此类里的每个detection方法读取的参数是一行Series

业务高频访问

In [ ]:
class HighFrequencyAccess:
    def __init__(self) -> None:
        pass

    def detection(self) -> int:
        pass

账号复用

In [4]:
class AccountReuse:
    def __init__(self) -> None:
        self.account_data = {}
        self.last_interval = None

    # detection()每次检查一条数据, for循环交给外部
    def detection(self, row):
        account = row['账号']
        ip = row['源IP']
        timestamp = row['日期时间']

        # 处理新账户
        if account not in self.account_data:
            self.account_data[account] = {'ips': set()}
        
        # 计算当前时间所在的时间区间
        current_interval = timestamp.floor('2T')  # 将时间戳向下取整到最近的两分钟时间区间
        # 如果是新的时间区间，清空account_data
        if current_interval != self.last_interval:
           self.account_data = {}
           self.last_interval = current_interval

        self.account_data[account]['ips'].add(ip)

        if len(self.account_data[account]['ips']) >= 3:
            print(f"账号 {account} 在时间 {timestamp} 存在账号复用行为")

非常用IP访问

In [8]:
class UncommonIp:
    def __init__(self):
        self.account_data = {}

    def detection(self, row):
            account = row['账号']
            ip = row['源IP']
            timestamp = row['日期时间']

            if account not in self.account_data:
                self.account_data[account] = {'ips_counts': {}, 'total_ops': 0}
            if ip not in self.account_data[account]['ips_counts']:
                self.account_data[account]['ips_counts'][ip] = 0
            else:
                self.account_data[account]['ips_counts'][ip] += 1
            self.account_data[account]['total_ops'] += 1

            if self.account_data[account]['total_ops'] > 100:
                uncommon_ips = [ip for ip, count in self.account_data[account]['ips_counts'].items() if count < 5]
                if len(uncommon_ips) > 0:
                    print(f"账号 {account} 在时间 {timestamp} 存在异常IP {uncommon_ips}")

非工作时间访问

In [ ]:
class OutOfHoursAccess:
    def __init__(self) -> None:
        pass

    def detection(self, data: pd.core.series.Series) -> int:
        pass

登录异常

In [ ]:
class LoginException:
    def __init__(self) -> None:
        pass

    def detection(self) -> int:
        pass